In [1]:
# Distribution ✨ jit ❇ Demo 💪 
%load_ext autoreload
%autoreload 2
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,4'
%env "WANDB_NOTEBOOK_NAME" "run" # ❕same as notebook
# from jax.config import config
# config.update('jax_disable_jit', True)

from functools import partial
import numpy as np
import re as regex
from pprint import pprint

import jax
from jax import pmap, grad
from jax import numpy as jnp
from flax.training.train_state import TrainState
from flax.core.frozen_dict import FrozenDict	

from pyfig import Pyfig
from hwat import FermiNet, sample, compute_ke_b, PotentialEnergy
from hwat import PotentialEnergy, sample, compute_ke_b
import wandb

from utils import flat_any

env: "WANDB_NOTEBOOK_NAME"="run" # ❕same as notebook


In [2]:
def collect_stats(k, v, new_d, p='tr', suf='', sep='/', sep_long='-'):
	depth = p.count('/')
	if depth > 1:
		sep = sep_long
	if isinstance(v, dict):
		for k_sub,v_sub in v.items():
			collect_stats(k, v_sub, new_d, p=(p+sep+k_sub))
	else:
		new_d[p+sep+k+suf] = v
	return new_d

def compute_metrix(d:dict, mode='tr'):
	pattern_ignore = ['Dense']
	fancy = dict(
		pe		= r'$V(X)',    				
		ke		= r'$\nabla^2',    		
		e		= r'$E',						
		log_psi	= r'$\log\psi', 			
		deltar	= r'$\delta_\mathrm{r}',	
		x		= r'$r_\mathrm{e}',
	)
	_d = {}
	for k,v in d.items():
		k = fancy.get(k, k)
		v = jax.device_get(v)
		if isinstance(v, FrozenDict):
			v = v.unfreeze()
		
		v_mean = jax.tree_map(lambda x: x.mean(), v) if not np.isscalar(v) else v
		v_std = jax.tree_map(lambda x: x.std(), v) if not np.isscalar(v) else 0.

		_d = collect_stats(k, v_mean, _d, p=mode, suf=r'_\mu$')
		_d = collect_stats(k, v_std, _d, p=mode, suf=r'_\sigma$')

	# return {k:v for k,v in _d.items() if not any([regex.match(k, pat) for pat in pattern_ignore])}
	return {k:v for k,v in _d.items() if not any([pat in k for pat in pattern_ignore])}


In [3]:
args = {'l_e':[4,],'n_u': 2,'n_b': 64, 'n_sv': 16, 'n_pv': 8, 'corr_len': 20, 'n_step': 1000, 'log_metric_step': 1,'exp_name':'demo-final'}
c = Pyfig(wandb_mode='online', args=args, get_sys_arg=False)


2022-11-30 19:34:24.205990: W external/org_tensorflow/tensorflow/compiler/xla/service/gpu/nvptx_compiler.cc:497] The NVIDIA driver's CUDA version is 11.4 which is older than the ptxas CUDA version (11.6.55). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


0 args unmerged: ✅
Path:  /home/amawi/projects/hwat/exp/demo-final/u2HRah9 ✅
System 
{'a': DeviceArray([[0., 0., 0.]], dtype=float32),
 'a_z': DeviceArray(4, dtype=int32, weak_type=True),
 'acc_target': 0.5,
 'corr_len': 20,
 'equil_len': 1000,
 'init_walker': functools.partial(<function init_walker at 0x7f5d16775d80>, n_b=64, n_u=2, n_d=2, center=DeviceArray([[0., 0., 0.]], dtype=float32), std=0.1),
 'l_e': [4],
 'n_b': 64,
 'n_d': 2,
 'n_e': 4,
 'n_u': 2}
Model 
{'compute_p_emb': functools.partial(<function compute_emb at 0x7f5d16774790>, terms=['xx']),
 'compute_s_emb': functools.partial(<function compute_emb at 0x7f5d16774790>, terms=['x_rlen', 'x']),
 'compute_s_perm': functools.partial(<function compute_s_perm at 0x7f5d16775510>, n_u=2),
 'n_det': 1,
 'n_fb': 16,
 'n_fb_out': 64,
 'n_pv': 8,
 'n_sv': 16,
 'terms_p_emb': ['xx'],
 'terms_s_emb': ['x_rlen', 'x']}


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xmax1. Use `wandb login --relogin` to force relogin


run:  xmax1/hwat/s8s35l1a ✅


In [4]:
part_sample = partial(sample, acc_target=c.data.acc_target)

rng = c.rng_init
x = c.data.init_walker(rng, n_b=c.data.n_b)
deltar = jnp.ones((c.n_device, 1), dtype=x.dtype)*0.02
print(f'Init: x {x.shape} rng {rng.shape} ✅')

@partial(jax.pmap, axis_name='b', in_axes=(0,0))
def create_train_state(rng, x):
    model = c.partial(FermiNet)  
    params = model.init(rng, x)
    state = TrainState.create(apply_fn=model.apply, params=params, tx=c.opt.tx)
    return state

state = create_train_state(rng, x)
print('Model ✅')

@partial(jax.pmap, axis_name='b', in_axes=(0,0,0,0))
def equil(rng, state, x, deltar):
    x, v_sam = part_sample(rng, state, x, deltar)
    return x, v_sam
print('Equil ✅')

wandb.define_metric("*", step_metric="eq/step")
for step in range(1, c.data.equil_len//c.data.corr_len):
    x, v_sam = equil(rng, state, x, deltar)
    rng, deltar, acc = v_sam['rng'], v_sam['deltar'], v_sam['acc']
    if not (step % c.log_metric_step):
        wandb.log({'eq/step':step, **v_sam})
print('Walkers ✅ Training Variables ✅')

@partial(pmap, in_axes=(0, 0, 0, 0))
def train_step(rng, state, x, deltar):
    x, v_sam = part_sample(rng, state, x, deltar)

    pe = partial(PotentialEnergy(a=c.data.a, a_z=c.data.a_z).apply, {})(x)
    ke = compute_ke_b(state, x)
    e = pe + ke

    def loss(_params):
        return (e * state.apply_fn(_params, x)).mean()
    
    grads = grad(loss)(state.params)

    v = dict(grads=grads, pe=pe, ke=ke, e=e, deltar=v_sam['deltar'],
                    rng=rng, x=x, acc=v_sam['acc'])
                    
    state = state.apply_gradients(grads=grads)
    return state, v

state, data = train_step(rng, state, x, deltar)
print('Train Step ✅')

print('Go seek: ', c.wandb_c.wandb_run_path)

Init: x (2, 64, 4, 3) rng (2, 2) ✅
Model ✅
Equil ✅


In [ ]:

# check model top to bottoms
# check acc and deltar
# test 1
# add r to electron inputs


wandb.define_metric("*", step_metric="tr/step")
for step in range(1, c.n_step+1):
    state, data = train_step(rng, state, x, deltar)
    rng, deltar, x = data['rng'], data['deltar'], data['x']

    if not (step % c.log_metric_step):
        metrix = compute_metrix(data)
        wandb.log({'tr/step':step, **metrix})
        m = ' '.join([f'{k} {v:.5f} ' for k,v in metrix.items() if 'E_' in k])
        print(f'Step {step} {m}')
    if not (step % c.log_state_step):
        ...

In [6]:
# 💓 
class tr_data:
    x:jnp.ndarray=x
    rng:jnp.ndarray=rng

# Likely needed for updating outside the loop, not sure
@jax.pmap
def update_model(state, grads):
  return state.apply_gradients(grads=grads)
print('Update ✅')


# add mutable states to the trian state where the parallelism is handled? 
from flax.training import train_state

class TrainState(train_state.TrainState):
  batch_stats: flax.core.FrozenDict[str, Any]

# how to include other variables
def loss_fn(params):
    outputs, new_model_state = state.apply_fn(
        {'params': params, 'batch_stats': state.batch_stats},
        inputs,
        mutable=['batch_stats'])
    loss = xent_loss(outputs, labels)
    return loss, new_model_state

  (loss, new_model_state), grads = jax.value_and_grad(
      loss_fn, has_aux=True)(state.params)
  new_state = state.apply_gradients(
      grads=grads,
      batch_stats=new_model_state['batch_stats'],
  )

class arg_cls(Pyfig):
    def __init__(_i):
        pass
arg_cls.data.n_e = 6
arg_cls.data.n_u = 6
arg_cls.data.n_b = 6
arg_cls.n_step = 20
arg_cls.log_metric_step = 5
arg_cls.exp_name = 'demo'
args = flat_any(arg_cls().d)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 44)